In [51]:
import numpy as np
from pathlib import Path
import pandas as pd

In [20]:
best_order = {}
for i in [3, 7, 8]:
    best_order[i] = ('fridge', 'hvac', 'dw', 'dr', 'mw')
best_order[12] = ('dw', 'hvac', 'fridge', 'dr', 'mw')

In [16]:
o = "\', \'".join(str(x) for x in best_order[mode])
o

"fridge', 'hvac', 'dw', 'dr', 'mw"

In [34]:
test_error = {}
dataset = 1
for mode in [3, 7, 8, 12]:
    test_error[mode] = {}
    for fold_num in range(5):
        test_error[mode][fold_num] = {}
        for lr in [0.001, 0.01, 0.1, 1.0]:
            test_error[mode][fold_num][lr] = {}
            
            
            f_name = '7_{}'.format(mode)
            o = "\', \'".join(str(x) for x in best_order[mode])
            directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/20000/0.0/".format(f_name, dataset, fold_num, lr)
            filename = "valid-error-[\'{}\'].npy".format(o)
            full_path = directory + filename
            my_file = Path(full_path)
#             print(filename)
            e = np.load(full_path).item()
            test_error[mode][fold_num][lr]= {}
            for iters in range(1000, 20001, 1000):
                test_error[mode][fold_num][lr][iters] = pd.Series(e[iters]).mean()

In [52]:
# find the best parameter for each mode
param = {}
for mode in [3, 7, 8, 12]:
    param[mode] = {}
    for fold_num in range(5):
        param[mode][fold_num] = {}
        
        min_value = np.inf
        for lr in [0.001, 0.01, 0.1, 1.0]:
            for iters in range(1000, 20001, 1000):
                if test_error[mode][fold_num][lr][iters] < min_value:
                    min_value = test_error[mode][fold_num][lr][iters]
                    param[mode][fold_num]['lr'] = lr
                    param[mode][fold_num]['iters'] = iters

In [54]:
# get the results
best_error = {}
for mode in [3, 7, 8, 12]:
    best_error[mode] = {}
    for fold_num in range(5):
        f_name = '7_{}'.format(mode)
        lr = param[mode][fold_num]['lr']
        o = "\', \'".join(str(x) for x in best_order[mode])
        directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/20000/0.0/".format(f_name, dataset, fold_num, lr)
        filename = "test-error-[\'{}\'].npy".format(o)
        full_path = directory + filename
        my_file = Path(full_path)
        e = np.load(full_path).item()
        best_error[mode][fold_num] =  pd.Series(e[iters])

In [67]:
pd.DataFrame(best_error[8]).mean(axis=1)

dr         89.563016
dw         15.715216
fridge     78.855606
hvac      216.633688
mw          8.860690
dtype: float64

In [68]:
pd.DataFrame(best_error[7]).mean(axis=1)

dr         83.784092
dw         15.557206
fridge     41.177983
hvac      157.226817
mw          8.372080
dtype: float64

In [69]:
pd.DataFrame(best_error[12]).mean(axis=1)

dr         70.206563
dw         17.305112
fridge     35.450528
hvac      196.851761
mw          8.676760
dtype: float64

In [70]:
pd.DataFrame(best_error[3]).mean(axis=1)

dr         59.744043
dw         24.273401
fridge     41.917410
hvac      182.370569
mw         47.127661
dtype: float64